### MNIST Classification using 2 layer CNN

In [1]:
#loading modules 
import numpy as np
import tensorflow as tf


/home/armughan/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/armughan/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
#loading hyperparameters
def vanilla_cnn(features,labels,mode):
    
    #reshaping features so that they can be used by convolutional layer input 
    x=tf.reshape(features['x'],[-1,28,28,1])
    
    conv1=tf.layers.conv2d(inputs=x,filters=32,kernel_size=[5,5],strides=1,padding='same',activation=tf.nn.relu)
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
#     bn1=tf.layers.batch_normalization(inputs=pool1,training=(mode==tf.estimator.ModeKeys.TRAIN))
    
    conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],strides=1,padding='same',activation=tf.nn.relu)
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    flat_pool2=tf.reshape(pool2, [-1, 7 * 7 * 64])
#     bn2=tf.layers.batch_normalization(inputs=pool2,training=(mode==tf.estimator.ModeKeys.TRAIN))
    
    
    dense = tf.layers.dense(inputs=flat_pool2,units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)


    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    
    

In [2]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [11]:
#Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=vanilla_cnn, model_dir="/tmp/mnist_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34d508f780>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=10,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.09680492 0.11414912 0.10211033 0.08581057 0.09767735 0.09343682
  0.11707465 0.08867776 0.10942014 0.09483835]
 [0.08891659 0.10614497 0.10318919 0.06992132 0.1048413  0.1087228
  0.1072963  0.08651544 0.11929644 0.10515565]
 [0.10178921 0.11040293 0.09314185 0.09703463 0.11348052 0.09788701
  0.10235678 0.09607411 0.09863956 0.08919341]
 [0.10070194 0.10142606 0.10752758 0.07605284 0.11008052 0.09286863
  0.10093386 0.08112945 0.1228539  0.10642519]
 [0.09647093 0.10269082 0.10381881 0.07954647 0.09030437 0.11353223
  0.09973568 0.09744451 0.11711562 0.09934051]
 [0.09799779 0.10558106 0.09072269 0.08700664 0.10598001 0.11310381
  0.10519858 0.0841469  0.11651756 0.09374503]
 [0.10324786 0.10271572 0.10007585 0.08261964 0.10586955 0.08977333
  0.11573295 0.09781488 0.10108495 0.10106527]
 [0.10381956 0.10478784 0.08649918 0.

INFO:tensorflow:loss = 2.310388, step = 1
INFO:tensorflow:probabilities = [[0.09773844 0.10916207 0.09247372 0.08683078 0.09930734 0.09631791
  0.11563867 0.08996423 0.1160899  0.09647697]
 [0.10939772 0.09894468 0.09486326 0.08747946 0.10693974 0.09641985
  0.11306795 0.10030274 0.10215784 0.0904268 ]
 [0.09709086 0.09346521 0.10059555 0.09085421 0.08779464 0.09341232
  0.115558   0.08820421 0.1166253  0.11639963]
 [0.08608963 0.11080696 0.10889985 0.08731267 0.09638807 0.09414993
  0.11896043 0.09010165 0.12135474 0.08593603]
 [0.10533725 0.09886985 0.08736953 0.08687958 0.09408493 0.0939986
  0.12105127 0.08659258 0.12758851 0.09822793]
 [0.10686485 0.09607201 0.09108594 0.08405464 0.10682733 0.1040678
  0.10741776 0.09379795 0.10671039 0.10310127]
 [0.09759935 0.10023836 0.10002578 0.08679286 0.10928535 0.10084059
  0.10063814 0.09451597 0.10944085 0.1006228 ]
 [0.1016054  0.09207746 0.11118047 0.08500756 0.11154211 0.09172869
  0.10482445 0.09325016 0.11787126 0.09091244]
 [0.0936

INFO:tensorflow:global_step/sec: 2.80093
INFO:tensorflow:probabilities = [[0.09793612 0.0962257  0.11115367 0.0818878  0.09297162 0.10022508
  0.10993107 0.1021276  0.11012963 0.0974117 ]
 [0.08957126 0.09798961 0.10112412 0.08771538 0.10995019 0.08814013
  0.11359506 0.0986196  0.1122695  0.1010251 ]
 [0.09561941 0.09884361 0.10552989 0.09189615 0.10831294 0.09993122
  0.11139662 0.08198203 0.11127087 0.09521727]
 [0.10198351 0.09825892 0.10169324 0.08741388 0.09776947 0.11451733
  0.10041776 0.09591731 0.10859473 0.09343381]
 [0.09970309 0.10364161 0.1099596  0.09237435 0.09788785 0.09703327
  0.10486018 0.08627716 0.10906813 0.09919478]
 [0.10063863 0.10355911 0.09364434 0.09637305 0.11524263 0.09376787
  0.09677564 0.09566902 0.10770549 0.09662427]
 [0.10948832 0.10899974 0.09606302 0.08274927 0.11549233 0.10136008
  0.09430794 0.08610589 0.11362786 0.09180557]
 [0.10553573 0.10047048 0.09825482 0.08772492 0.10173579 0.09089082
  0.11719365 0.08704392 0.11243466 0.09871518]
 [0.100

INFO:tensorflow:loss = 2.2682858, step = 101 (35.714 sec)
INFO:tensorflow:probabilities = [[0.1031023  0.09407872 0.10808873 0.08366242 0.1039631  0.09505536
  0.10303692 0.09764601 0.12290314 0.08846338]
 [0.09372625 0.10118309 0.10628553 0.09010734 0.10397188 0.0924783
  0.11357612 0.10120303 0.10010264 0.09736581]
 [0.14344428 0.09623795 0.08442547 0.07898656 0.09005144 0.109546
  0.10956334 0.07290903 0.10413968 0.11069626]
 [0.10519885 0.08720658 0.11037847 0.08923495 0.09939674 0.09480662
  0.10934954 0.10003561 0.09901507 0.10537753]
 [0.10319905 0.09621307 0.09211194 0.09458458 0.09355317 0.08972944
  0.1034009  0.10477693 0.12535058 0.09708041]
 [0.09671236 0.1041273  0.10661875 0.09117924 0.09996857 0.09781258
  0.10114236 0.08386387 0.11325187 0.10532311]
 [0.10917783 0.09585986 0.10763881 0.07874784 0.08335115 0.08452719
  0.11653532 0.0857935  0.14498785 0.0933807 ]
 [0.10457272 0.10311002 0.10308123 0.08278603 0.10268095 0.09956455
  0.10137265 0.091754   0.10507942 0.105

INFO:tensorflow:global_step/sec: 2.81377
INFO:tensorflow:probabilities = [[0.09927043 0.10498346 0.10189655 0.0887652  0.10013343 0.08907568
  0.10983486 0.09268285 0.11243787 0.1009196 ]
 [0.10925923 0.09324035 0.09379659 0.09338017 0.10634414 0.08948135
  0.10533827 0.1027385  0.10364358 0.10277779]
 [0.09292307 0.08480577 0.10488931 0.08733277 0.10810129 0.1037903
  0.10403308 0.09497103 0.11825462 0.10089885]
 [0.09888269 0.09836299 0.11124178 0.08669709 0.09076633 0.09377386
  0.10661421 0.08930999 0.13382776 0.09052327]
 [0.1070433  0.09650747 0.10504978 0.08981297 0.1009619  0.09631966
  0.09894118 0.0975974  0.10931925 0.09844705]
 [0.09883127 0.08644602 0.09804773 0.09916458 0.09252805 0.11662824
  0.11891569 0.0904816  0.10422989 0.09472694]
 [0.10562671 0.08918158 0.09261365 0.10179879 0.08804886 0.10708888
  0.09516153 0.10176051 0.12126756 0.09745199]
 [0.10522724 0.0928978  0.11863656 0.09391642 0.08321504 0.09365705
  0.11061742 0.08613347 0.12409312 0.09160591]
 [0.1132

...

INFO:tensorflow:loss = 0.4394609, step = 3301 (35.805 sec)
INFO:tensorflow:probabilities = [[0.00091627 0.92825234 0.00300035 0.01499089 0.00280943 0.00264008
  0.00831553 0.00725542 0.01997415 0.01184556]
 [0.00153338 0.01455177 0.03414773 0.21324866 0.00333953 0.04099474
  0.01729633 0.015683   0.627253   0.03195189]
 [0.00397914 0.00000995 0.95815253 0.00629766 0.00044331 0.0000502
  0.02983684 0.00001572 0.00098616 0.00022856]
 [0.00201582 0.00008925 0.00110059 0.00191534 0.13130765 0.00460307
  0.00437273 0.06816474 0.01483749 0.7715934 ]
 [0.00002172 0.00000023 0.00000266 0.00021186 0.00005212 0.00024375
  0.00000088 0.9965785  0.00002063 0.00286762]
 [0.00108706 0.00049036 0.00660102 0.00245446 0.01202866 0.00241251
  0.94718856 0.00223565 0.01078478 0.01471684]
 [0.00268693 0.00000528 0.00106877 0.00860289 0.00752877 0.7203586
  0.0390443  0.00040382 0.20089704 0.01940339]
 [0.00605528 0.00024362 0.00643362 0.00443754 0.07582171 0.00855019
  0.00185729 0.10665575 0.19764553 0.5

INFO:tensorflow:global_step/sec: 2.70881
INFO:tensorflow:probabilities = [[0.95836794 0.00000002 0.01548249 0.00009049 0.00000013 0.02274997
  0.00026    0.00004278 0.00299581 0.00001031]
 [0.0000207  0.07064909 0.00351111 0.0349782  0.02389232 0.00120736
  0.00022945 0.19559124 0.0544899  0.61543065]
 [0.00002286 0.00224238 0.00003201 0.00049977 0.02856813 0.00034776
  0.00005676 0.07149853 0.00560332 0.8911284 ]
 [0.00525427 0.00000008 0.98293316 0.00006381 0.00003745 0.00000715
  0.01111383 0.00000003 0.00058049 0.00000969]
 [0.01029459 0.00032147 0.00027892 0.01802818 0.02761991 0.5165863
  0.00151381 0.05202714 0.12209413 0.2512356 ]
 [0.98882514 0.00000016 0.00019628 0.00027046 0.00003733 0.00959468
  0.00086868 0.00001107 0.00016431 0.00003198]
 [0.80005825 0.00001408 0.01488527 0.0055867  0.00013977 0.0643958
  0.00174811 0.00306559 0.10969572 0.00041078]
 [0.00020817 0.00004291 0.01138447 0.00024249 0.01791349 0.00040493
  0.96073556 0.00004786 0.00895902 0.00006115]
 [0.05832

INFO:tensorflow:loss = 0.2795963, step = 3401 (36.915 sec)
INFO:tensorflow:probabilities = [[0.00000288 0.00010547 0.99436104 0.00114078 0.00001737 0.00004901
  0.00267262 0.00011696 0.00137606 0.00015772]
 [0.00026363 0.00001424 0.00169399 0.00040997 0.9815663  0.0002006
  0.00161183 0.00079185 0.00237019 0.01107744]
 [0.00062232 0.00035165 0.011342   0.125478   0.02863181 0.08918954
  0.04896031 0.00062922 0.6912292  0.00356593]
 [0.00192875 0.00451354 0.00139767 0.05534308 0.00032236 0.07347547
  0.00117247 0.00052232 0.8575076  0.00381675]
 [0.12115424 0.00009863 0.08131508 0.09527181 0.00059061 0.46310306
  0.06370053 0.00002505 0.17438613 0.00035486]
 [0.00000275 0.9731151  0.00253119 0.00270982 0.00006325 0.00038108
  0.00055171 0.00030691 0.01996433 0.00037396]
 [0.00003905 0.00013328 0.00033143 0.00001804 0.9839998  0.00243892
  0.00866675 0.0000717  0.00186717 0.0024338 ]
 [0.1970371  0.00010437 0.01329674 0.08709107 0.02059937 0.562768
  0.09932493 0.00014997 0.01942099 0.00

INFO:tensorflow:Saving checkpoints for 3452 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:global_step/sec: 2.54232
INFO:tensorflow:probabilities = [[0.9662905  0.         0.00000332 0.00067246 0.00000002 0.03301158
  0.00000119 0.00000183 0.00001877 0.00000051]
 [0.00043586 0.00067116 0.0000418  0.0076237  0.0005102  0.00154921
  0.0000435  0.8788779  0.00144619 0.10880041]
 [0.00017465 0.00007196 0.00003955 0.00030202 0.3096945  0.00123447
  0.00058737 0.02519059 0.00904045 0.65366447]
 [0.0691969  0.00000702 0.00025664 0.03069553 0.0000149  0.89553297
  0.00042511 0.00052979 0.00305809 0.00028297]
 [0.00104331 0.00147755 0.08086781 0.00204124 0.66740596 0.00166282
  0.20080416 0.01167068 0.01307271 0.01995374]
 [0.00000034 0.00001203 0.99937916 0.00036555 0.00000007 0.00000028
  0.00019624 0.00000093 0.00004534 0.00000015]
 [0.00001129 0.00558277 0.0017751  0.00001903 0.00570866 0.00011171
  0.986233   0.00001048 0.0005186  0.00002923]
 [0.00000892 0.00000619 0.00000172 0

INFO:tensorflow:loss = 0.52665794, step = 3501 (39.341 sec)
INFO:tensorflow:probabilities = [[0.00123563 0.00521552 0.00317292 0.82363015 0.00000551 0.02137349
  0.00006922 0.00140668 0.14356291 0.00032797]
 [0.00020555 0.8798307  0.00765881 0.06902974 0.00199269 0.00728387
  0.00746479 0.00335563 0.01522482 0.00795338]
 [0.00345439 0.00034593 0.89890206 0.01859781 0.00012046 0.00195814
  0.00081835 0.00117737 0.07345196 0.00117349]
 [0.0000087  0.00002214 0.00003162 0.00023788 0.95433503 0.00531208
  0.00382179 0.00247777 0.00316448 0.03058858]
 [0.00183489 0.01820392 0.00671571 0.02524458 0.00125739 0.04013478
  0.01951019 0.01673445 0.78015167 0.09021251]
 [0.00958604 0.11032059 0.00360272 0.0390691  0.0326662  0.47087145
  0.11188547 0.00253367 0.20492841 0.01453636]
 [0.00000548 0.98950815 0.00039993 0.00730382 0.00003584 0.00006052
  0.00007884 0.00096654 0.0012923  0.00034863]
 [0.04659784 0.0007036  0.00685623 0.8887095  0.0000081  0.02183281
  0.00055958 0.00059549 0.03383409 

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)